[View in Colaboratory](https://colab.research.google.com/github/KushalVenkatesh/Sentiment_Analysis_Customer_Reviews_TestPilot/blob/master/updated_scrape_reviews.ipynb)

# Scrape reviews

In [0]:
!pip install request


distributed 1.21.8 requires msgpack, which is not installed.
grin 1.2.1 requires argparse>=1.1, which is not installed.


In [0]:
!pip install python-requests

  Could not find a version that satisfies the requirement python-requests (from versions: )
No matching distribution found for python-requests


In [0]:
import urllib
urllib.urlretrieve

<function urllib.urlretrieve>

In [0]:
#urllib.urlopen(url)

In [0]:
import numpy as np
from bs4 import BeautifulSoup
import urllib
import datetime
import os
import json
import time

In [0]:
!pip install urllib

  Could not find a version that satisfies the requirement urllib (from versions: )
No matching distribution found for urllib


## Scrape home page

In [0]:
url_rebtel = 'https://www.trustpilot.com/review/www.rebtel.com'
url_rebtel_page = url_rebtel + '?page='

In [0]:
with urllib.request.urlopen(url_rebtel) as url:
    homepage = url.read()

In [0]:
soup = BeautifulSoup(homepage, 'html.parser')
# print(soup.prettify())

In [0]:
# Explore content of home page
# list(soup.children)
# soup.head
print (soup.prettify()[0:200])

<!DOCTYPE html>
<html lang="en">
 <head>
  <!--[if IE]>
<meta http-equiv="X-UA-Compatible" content="chrome=1">
<![endif]-->
  <title>
   Rebtel Reviews | Read Customer Service Reviews of www.rebtel.co


In [0]:
# Parse content/reviews of home page
reviews = soup.find_all('div', class_='review-stack')
container = soup.find_all('div', class_='reviews-container')
len(reviews)

20

In [0]:
# Parse total number of review pages
pagination = soup.find_all('a', class_='pagination-page')
page_num = [page['data-page-number'] for page in pagination]
page_num = [int(page) for page in page_num if page != 'next-page']
page_max = max(page_num)
page_max

157

## Scrape all pages

In [0]:
data = []  # reviews is a list of dictionaries

# Iterate over webpages
for page in range(1, page_max + 1):
    
    # If first page, take url of homepage; otherwise, url + page
    if (page == 1):
        url_complete = url_rebtel
    else:
        url_complete = url_rebtel_page + str(page)
    
    # Get content of webpage
    with urllib.request.urlopen(url_complete) as url:
            page_content = url.read()        
    soup = BeautifulSoup(page_content, 'html.parser')
    reviews = soup.find_all('div', class_='review-stack')
    
    # Iterate over reviews in page
    for i in range(0, len(reviews)):  
        # Parse user id
        user_id = reviews[i].find('a', class_='consumer-info')['href'].strip()
        user_id = user_id.replace('/users/', '')
        print(user_id)
        
        # Parse user name
        user_name = reviews[i].find('h3', class_='consumer-info__details__name').get_text().strip()
        print(user_name)
        
        # Parse review title
        review_title = reviews[i].find('h2', class_='review-info__body__title')
        review_title = review_title.get_text().strip()
        print(review_title)
        
        # Parse review content
        review_text = reviews[i].find('p', class_='review-info__body__text').get_text().strip()
        print(review_text)
        
        # Parse review answer
        review_answer = reviews[i].find('div', class_='company-reply__content__body')
        if (review_answer is not None):
            review_answer = review_answer.get_text().strip()
        else:
            review_answer = None
        print(review_answer)
        
        # Parse review stars
        review_stars = int(reviews[i].find('div', class_='social-share-network social-share-network--twitter')['data-status'].split()[-5])
        
        # Parse review date
        review_date = reviews[i].find('time', class_='ndate')['datetime'].strip()
        review_date = review_date.replace('.000+00:00', '')
        review_date = review_date.replace('T', ' ')
        
        # Parse whether review was verified by Rebtel
        review_verify_text = reviews[i].find('div', class_='review-verified')
        if (review_verify_text is not None):
            review_verify = 'Verified order' in review_verify_text.get_text()
        else:
            review_verify = False        
        
        # Store data
        data.append({  
            'user_id': user_id,
            'user': user_name,
            'title': review_title,
            'text': review_text,
            'answer': review_answer,
            'stars': review_stars,
            'date': review_date,
            'verify': review_verify
        })
    

In [0]:
len(data)

3124

## Save data

In [0]:
#!ls
#%cd rebtel_sentiment_analysis
!ls
#%cd ..

code  data  docs  environment.yml  notebooks  README.md


In [0]:
today = time.strftime('%Y%m%d')
file_name = 'data/rawdata_' + today + '.json'

In [0]:
with open(file_name, 'w') as outfile:  
    json.dump(data, outfile, indent = 2)

In [0]:
!ls


code  data  docs  environment.yml  notebooks  README.md


In [0]:
pwd

'/content/rebtel_sentiment_analysis'